# Bisection method
Ma devo eseguire un fit anche per trovare q? O basta il fatto che all'aumentare delle misure si avvicina a 1? Perchè se faccio il fit non viene bene

Puoi anche fare un grafico in cui metti i punti stimati f(x_found)

In [ ]:
using Markdown
using Plots
using LaTeXStrings
using Printf
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\linear_systems.jl")
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\non_linear_roots.jl")
function autosave(fig, filename)
    path = "C:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\relazione\\immagini"
    savefig(fig, joinpath(path, filename))
end

In [ ]:
colors = ["#0F2080",  # blu scuro
           "#F5793A",  # arancione
           "#A95AA1",  # viola
           "#85C0F9"]  # azzurro

**(a)** Use the bisection method to find solutions for $x^3 - 7x^2 + 14x - 6 = 0$ on $[0,1]$.

In [ ]:
function polynomial(x)
    return x^3 - 7x^2 + 14x - 6
end

In [ ]:
a = 0
b = 1

root, steps = bis_met_steps(polynomial, a, b)
display(Markdown.parse("Calculated root: "))
display(root)
display(Markdown.parse("Function evaluated in calculated root: "))
display(polynomial(root))
display(Markdown.parse("Number of steps: "))
display(length(steps))

In [ ]:
x = [i for i in a:0.01:b]
y = polynomial.(x)

fig = plot(xlabel=L"x", ylabel=L"f(x)",
           framestyle=:box,
           grid=true, gridalpha=0.5,
           legend=:bottomright,
           xticks=a:0.2:b,
           legendfontsize = 10
          )

plot!(fig, x, y,
      label = L"x^3 - 7x^2 + 14x - 6",
      color = colors[1]
     )
plot!(fig, x, fill(0, length(x)), 
      label = "",
      color = colors[2]
     )
scatter!(fig, steps, fill(0, length(steps)),
         color = colors[3],
         label = L"Estimated\ roots",
         marker = :circle, 
        )

autosave(fig, "3221.pdf")
display(fig)

**(b)** Study the convergence of the bisection method to the root found in **(a)**. (You can take as an approximation for the root $r=x_{n_{\rm max}}$, where $n_{\rm max}$ is the index corresponding to the last bisection iteration.) What is its order of convergence $q$? Can you estimate the asymptotic error constant $C$? We suggest studying $d_n=-\log_{10}|x_{n}-r|$ as a function of $n$.

Vale per il metodo di bisezione: $ |m_n-r|< 2^{-(n+1)} (b_0-a_0) $ \
Ad ogni iterazione l'intervallo si dimezza, al passo successivo la stima è nella metà dell'intervallo a cui stava la stima precedente. In simboli: $|m_{n+1}-r| <\frac{|m_n-r|}{2}$. \
Dalla relazione sopra si ricava che $\frac{|m_{n+1}-r|}{|m_n-r|} < \frac{1}{2}$. Passando al limite: \
$\lim_{n\to\infty} \frac{|m_{n+1}-r|}{|m_n-r|} = \frac{1}{2}$, da cui ricaviamo che $q$, ordine di convergenza, è 1, mentre C, la costante di errore asintotico, è $\frac{1}{2}$ \
\
Verifichiamolo numericamente, ricordando le relazioni: 
$$
    d_{n+1}\approx q\,d_{n} -\log_{10}|C|
    \quad
    \Rightarrow
    \quad
    {d_{n+1}\over d_n}\approx q -{\log_{10}|C|\over d_n}\ \overset{n\to\infty}{\longrightarrow}\ q
$$

# q and C evaluation

In this case I can evaluate q it by plotting ${d_{n+1}\over d_n}$ and see its value for $n\to\infty$. 

In [ ]:
#This cell is subtle because plays with the length of vectors.
#The first subtlety is redfining the steps, because I don't want the estimated root to be a step, or else the number of digits diverges
#The second one is to define the number of digits for every step except the last one, because the iteration uses the n+1 step
#In the end, I need to correct q_approx's length, by adding the last element manually
a = 0
b = 1
pol_zero, zero_steps = bis_met_steps(polynomial, a, b)

#This step isn't useful from a pratical point of view, but is theoretical
#It says: we have estimated the root with bis_met_steps (pol_zero), now we are approximating the real one (r) with it
r = pol_zero        

zero_steps = zero_steps[1:(end-1)]          # I do not consider the entire zero_steps vector because the last entry is exactly r
# vector of digit numbers for each step
#Contains every digit number of the steps, except for the last one
d = -log10.(abs.(zero_steps[1:end-1] .- r))         
q_approx =  []
#itero per tutti gli elementi di d, considerando che devo guardare d_n e d_(n+1)
for i  in 1:(length(d)-1)
    push!(q_approx, d[i+1]/d[i])
end
push!(q_approx, (-log10.(abs.(zero_steps[end-1] .- r)))/d[end])

println(length(zero_steps))
println(length(d))
println(length(q_approx))

Migliora il plot aggiungendo i punti per ogni n

In [ ]:
#Plot cell
n = [i for i in 1:(length(d))]

fig1 = plot(xlabel=L"n", ylabel=L"Digits\ ratio",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            legend=:topright,
            xticks=1:4:length(n),
            legendfontsize = 10
            )

plot!(fig1, n, q_approx,
      label=L"d_{n+1}/d_n",
      marker=:circle,
      color = colors[1],
     )

plot!(fig1, n, fill(1, length(n)),
      label=L"Expected\ order\ of\ convergence",
      color  = colors[2],
     )
     

display(fig1)

From the graph, I notice that to determine q, I should use the last points to perform the fit. \
$y = const$

In [ ]:
start_point = 29
n_qfit = n[start_point:end]
q_approx_qfit = q_approx[start_point:end]
#design matrix
D = hcat(ones(length(n_qfit)))
q = least_sq(D, q_approx_qfit)
result_text = @sprintf("Estimated q = %.2f", q[1])
display(Markdown.parse(result_text))

In [ ]:
#plot cell
fig2 = plot(xlabel=L"n", ylabel=L"Digits\ ratio",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            legend=:bottomright,
            xticks=0:2:length(n), yticks=0.8:0.05:1.2,
            xrange=(start_point-1, length(n)+1), yrange=(0.79, 1.13),
            legendfontsize = 10
            )   

plot!(fig2, n[start_point:end], q_approx[start_point:end],
      label="",
      color = colors[1],
      marker=:circle,
     )

plot!(fig2, 1:1:length(n)+1, fill(1, length(n)+1),
      label="",
      color  = colors[2],
     )

plot!(fig2, 1:1:length(n)+1, fill(q[1], length(n)+1),
      label= L"Estimated\ q",
      color = colors[3],
     )

display(fig2)


In [ ]:
fig_merge = plot(fig1, fig2, layout = (2, 1), size = (800, 500),
    )

autosave(fig_merge, "3222.pdf")

C value must be evalueated with a fit, remembering the fact that $ |x_{k+1} - r|= C |x_{k} - r|$.  \
Iterating this process gives us $ |x_{k+1} - r|= C^{k+1} |x_{0} - r|$. By evaluating the natural logaritm we can make this equation linear and fit the data with C as a parameter. \
Fit model: $log |x_{k+1}-r| = (k+1)log(C) + log|x_0 - r|   \rightarrow   y(x) = x log(C) + A$ 

In [ ]:
# For this fit I just need x_k, so zero_step
pol_zero, zero_steps = bis_met_steps(polynomial, a, b)
r = pol_zero        
zero_steps = zero_steps[1:(end-1)]  
x = [k for k in 1:1:length(zero_steps)]
y = log.(abs.(zero_steps .- r))
# Design matrix
D = hcat(x, fill(1.0, length(x)))
coeff = least_sq(D, y)

# Display results in a single Markdown block
C = exp(coeff[1])
A = coeff[2]
result_text = @sprintf("Linear fit coefficients are: \nC = %.5f\nA = %.5f", C, A)
display(Markdown.parse(result_text))

In [ ]:
#plot cell
fig = plot(yscale=:log10,
            xlabel=L"n", ylabel=L"|x_n - r|",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            legend=:topright,
            legendfontsize = 10
            )

plot!(fig, x, exp.(y),
      label=L"|x_n - r|",
      marker=:circle,
      markersize=3.5,
      color = colors[1],
     )            

          
plot!(fig, x, exp.(coeff[1] .* x .+ coeff[2]),
        label=latexstring("y(n) = n log_{10}(", @sprintf("%.3f", C) ,") ", @sprintf("%.3f", A)),
        color = colors[2],
        linestyle=:dash,
        linewidth=2,
        )

autosave(fig, "3223.pdf")
display(fig)

# q and C fit
q and C could be also calculated with another. The relation is $y(d_n)\approx q -{\log_{10}|C|\over d_n}\ \overset{n\to\infty}{\longrightarrow}\ q$. I need to make this a linear relation in the coefficients, so instead of searching for $C$ I will search for $k = -log_{10}|C|$. \
The next step is to create the design matrix and using least_sq function, defined in the linear_systems.jl library. \
The design matrix has the first column of ones, the second column with $\frac{1}{x}$ function \
NOTE: it is not a really accurate method

In [ ]:
order = sortperm(d)
d_sorted = d[order]
q_approx_sorted = q_approx[order]
#As data we have on the x axis d, on the y axis d(n+1)/dn
A = hcat(ones(length(d_sorted)), [1.0/i for i in d_sorted])
coeff = least_sq(A, q_approx_sorted)
q = coeff[1]
C_module = exp10(-coeff[2]) 
println("Order of convergence q: ", q)
println("Expected: ", 1)
println("Asymptotic error constant: ", C_module)
println("Expected: ", 0.5)

In [ ]:
#Plot cell
fig = plot(title="Least square fit",
            xlabel="digits", ylabel=L"\frac{d_{n+1}}{d_n}",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            legend=:topright,
            )
plot!(fig, d_sorted, q_approx_sorted,
        label=L"\frac{d_{n+1}}{d_n}",
         )
plot!(fig, d_sorted, q .- log10.(C_module)./d_sorted,
        label="Least square fit",
        )
display(fig)
